# Trainver v5 code validation

Simple minimal training runs, to validate v5 code

> Important note: These example focuses only on how to configure your dataset, and does not properly perform checkmarking - for trainer configurations refer to the training notebooks

In [16]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=False
WANDB_PREFIX="trainer-v5wavenet-validation L6-D512"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5wavenet/"))
INFERENCE_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5wavenet/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("INFERENCE_DIR:", INFERENCE_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: False
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment/notebook/trainer-x-validation
INFERENCE_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v5wavenet
TRAINER_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v5wavenet
PROJECT_DIR: /home/picocreator/rwkv-proj/picocreator-memory-experiment


## Intial setup

Before we go into the dataset setup, lets perform an initial setup for all the folders we need, and a small toy model which we would use throughout the various examples within this notebook.

In [11]:
# Setup the folders we will need
!mkdir -p ../../model/
!mkdir -p ../../datapath/
!mkdir -p ../../checkpoint/

# Initialized a simple L6-D512 model
!cd "{TRAINER_DIR}" && python3 ./init_model.py --n_layer 6 --n_embd 512 --vocab_size neox --skip-if-exists ../model/L6-D512-v5wavenet-neox-init.pth


[2023-08-07 13:17:53,853] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
---- Initializing model ----
No of layers: 6
Embedding size: 512
Output model path: ../model/L6-D512-v5wavenet-neox-init.pth
Vocab size: 50277
---- ----- ----
Model exists, skipping init_model


## Quick train for v5

Preload and train the mini-v5 model

In [17]:
# Lets preload the requried dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py ../notebook/trainer-x-validation/mini-v5wavenet-enwiki.yaml 

[2023-08-07 13:21:15,553] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Found cached dataset parquet (/home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 796.34it/s]
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-e6926d59afde2d0b_*_of_00016.arrow
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-59b52cf35738a88f_*_of_00016.arrow
Loading cached processed dataset at /home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_

In [21]:
# Validate the dataset is working, by doing a quick training run
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c ../notebook/trainer-x-validation/mini-v5wavenet-enwiki.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (full, train-ctx=4096, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"

[2023-08-07 13:39:06,328] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:484: UserWarning: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '../notebook/trainer-x-validation/mini-v5wavenet-enwiki.yaml', '--trainer.logger.init_args.name=trainer-v5wavenet-validation L6-D512 (full, train-ctx=4096, data-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto'], args=['fit', '-c', '../notebook/trainer-x-validation/mini-v5wavenet-enwiki.yaml', '--trainer.logger.init_args.name=trainer-v5wavenet-validation L6-D512 (full, train-ctx=4096, data-ctx=4096, deepspeed_stage

In [20]:
# Lets convert the model
!cd "{TRAINER_DIR}" && \
    python3 export_checkpoint.py \
        ../checkpoint/trainer-validation/mini-v5wavenet-enwiki/last.ckpt/ \
        ../model/mini-v5wavenet-enwiki.pth

[2023-08-07 13:38:42,534] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Traceback (most recent call last):
  File "/home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v5wavenet/export_checkpoint.py", line 623, in <module>
    convert_zero_checkpoint_to_fp32_state_dict(args.checkpoint_dir, output_file)
  File "/home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v5wavenet/export_checkpoint.py", line 537, in convert_zero_checkpoint_to_fp32_state_dict
    state_dict = get_fp32_state_dict_from_zero_checkpoint(checkpoint_dir, tag)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/picocreator/rwkv-proj/picocreator-memory-experiment/RWKV-v5wavenet/export_checkpoint.py", line 516, in get_fp32_state_dict_from_zero_checkpoint
    raise ValueError(f"Unable to find 'latest' file at {latest_path}")
ValueError: Unable to find 'latest' file at ../checkpoint/trainer-validation/mini-v5wavenet

In [ ]:
# And run the dragon test
!cd "{TRAINER_DIR}" && \
    python3 dragon_test.py ../model/mini-v5wavenet-enwiki.pth